# "No Touch" Reporting 
The following is a example of how SQL and Python can be leveraged to deliver reporting that requires no code changes

Connect to Teradata -> Run Queries -> Create Dataframe -> Create Excel -- Send Via Email and Save Draft

In [ ]:
# Import libraries
import os
import getpass
import teradata
import sys
import pandas as pd 
import logging  
logging.getLogger().setLevel(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import win32com.client as client

# Define Date Variables and Set Working Directory

In [ ]:
# Get dates 
now = datetime.now()
month_name = now.strftime("%b")
month = now.month
year = str(datetime.now().year)
year_abbrev = year[-2:]

# Set working directory
work_dir = '\\\some_drive_name\some_folder'
os.chdir(work_dir)

# Enter Login Credentials

In [ ]:
# Input credentials
user = input('Enter Username: ')
auth = input('Enter Auth type (LDAP for UID, TD2 for NPID): ')
passw = getpass.getpass()

# Connect to Teradata

In [6]:
# Connect
udaExec = teradata.UdaExec(appName="test", version="2.0", logConsole=False)
session = udaExec.connect(method="odbc", system='',
                          username=user, password=passw, authentication=auth,
                          driver="Teradata Database ODBC Driver")

# Run Query to Determine Quarter

This is set up so that only the previous complete Quarter will return until the month the report is meant to be run.


In [7]:
# Create volatile table to get dates for each quarter
session.execute(
"""
CREATE VOLATILE TABLE TMP_DT AS (
SELECT
Stuff
FROM YOUR_DB.YOUR_TABLE
) WITH DATA ON COMMIT PRESERVE ROWS;
"""
)


# Read in SQL and Create Dataframe

In [8]:
# Open script (save some space by not rewriting)
with open ('Auto_Script.sql','r') as sql_file:
    script = sql_file.read()

In [9]:
# read output
df = pd.read_sql(script,session)


# Define Quarter and Create Excel

In [12]:
# Determine Qtr 
if 4 <= month <=6:
    qtr='Q1'
elif 7 <= month <=9: 
    qtr='Q2'
elif 10 <= month <=12:
    qtr='Q3'
elif 1 <= month <=3:
    qtr='Q4'

In [13]:
qtr

'Q1'

In [14]:
print('Creating Excel...')

# Create Excel and Format
filename=f'Stuff_file_{year}_{qtr}.xlsx'
writer = pd.ExcelWriter(filename, engine='xlsxwriter')



Creating Excel...


In [15]:
df.to_excel(writer, sheet_name= qtr, index=False)

workbook = writer.book

worksheet1 = writer.sheets[qtr]


# Format Excel and Save

In [16]:
# Set first row formatting
row_all = workbook.add_format({'bold': True, 'align': 'center'})

# Set Column formatting
column_all = workbook.add_format({ 'align': 'left'})
worksheet1.set_column('A:AR', 15, column_all)
worksheet1.set_column('B:B', 22, column_all)
worksheet1.set_column('D:D', 25, column_all)

# Add currency format to Paid
currency_format = workbook.add_format({'num_format': '$#,##0.00'})
worksheet1.set_column('AE:AR', 15, currency_format)

0

In [17]:
print('Saving...')
writer.save()

Saving...


Create Email, Add Attachment, and Save Draft

In [20]:
outlook = client.Dispatch("Outlook.Application")

message = outlook.CreateItem(0)

message.Display()

message.To = 'grumpy_statistician@gmail.com'

message.CC = 'Test; Test'

message.Subject = f'Stuff{year_abbrev} {qtr}'

message.HTMLBody = f'''

<p style="font-size:15px"> Howdily-doodily,<br><br>


Please see attached for CY{year_abbrev} {qtr}.<br><br>


Thanks!<br> </p>

''' + message.HTMLBody

In [21]:
path_file = os.path.join(work_dir,filename)

message.Attachments.Add(path_file)

message.Saveas(f"{work_dir}//{month_name}_{year}-Report_Email.msg")

print('Email Draft Saved')

Email Draft Saved
